# Plot maps where discharge algorithms work

Confluence summit at U Mass, April 2024

## Set up Libraries and Directories

In [ ]:
import os,sys
import netCDF4
from pathlib import Path
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# set up SWOT DAWG viz
sys.path.append('/nas/cee-water/cjgleason/stevec/SWOTdawgDISTRO/notebooks/Umass')
from swotdawgviz.swotdawgviz import io as sdvio
from swotdawgviz.swotdawgviz import maps as sdvm

In [ ]:
os.getcwd()

In [ ]:
# set up directories
DataDir=Path('.')

## Open files

In [ ]:
results = netCDF4.Dataset("/nas/cee-water/cjgleason/SWOT_Q_UMASS/na_sword_v16_SOS_results_EOD_day1.nc", format="NETCDF4")

print("Results Group")
print(results, "\n")


## Parse Ohio basin in SoS

The Ohio is L4=7426

In [ ]:
reaches = results.groups['reaches']
print("Reaches Group")
print(reaches, "\n")

reachids=reaches['reach_id'][:]

reachid_strs=[]

reachids_oh=[]
reachids_int_oh=[]

for reachid in reachids:
    if str(reachid)[0:4]=='7426':
        reachids_oh.append(str(reachid))
        reachids_int_oh.append(int(reachid))

In [ ]:
reachids

## Explore integrator data stored in SoS

In [ ]:
moi = results.groups['moi']
print("MOI Group")
print(moi, "\n")

In [ ]:
print("HiVDI in MOI")
print(moi['hivdi'], "\n")

In [ ]:
print("Basin scale discharge from HiVDI in MOI")
print(moi['hivdi']['qbar_basinScale'], "\n")

In [ ]:
moi['hivdi']['qbar_basinScale'][14954].data

## Initialize SWOT DAWG Viz map

In [ ]:
import geopandas as gpd

rch = gpd.read_file("/nas/cee-water/cjgleason/miked/umass_workshop/sword_shp/na_sword_reaches_hb74_v16.shp")
rch.plot()
print(len(rch))

In [ ]:
rch[rch.reach_id.isin(reachids.data)].plot()

In [ ]:
# Open the priors file:
priors = netCDF4.Dataset("/nas/cee-water/cjgleason/SWOT_Q_UMASS/na_sword_v16_SOS_priors.nc", format="NETCDF4")

gauge_reach = priors["USGS"]["USGS_reach_id"][:]
print("Gauge reach identifiers:")
print(gauge_reach)

reach_overlap = np.intersect1d(gauge_reach, reachids.data)
print("Overlapping reaches:")
print(reach_overlap)
reach_overlap

In [ ]:
len(reach_overlap)

In [ ]:
gauged_reaches = rch[rch.reach_id.isin(reach_overlap)]
print(len(gauged_reaches))
gauged_reaches.plot()

In [ ]:
gauged_reaches

results#["hivdi"]["Q"]

In [ ]:
# create swotdawgviz map with just the reaches in RL mapped
sword_hb_reaches = sdvio.SwordShapefile("/nas/cee-water/cjgleason/miked/umass_workshop/sword_shp/na_sword_reaches_hb74_v16.shp",reachids_int_oh)
rmap = sdvm.ReachesMap(sword_hb_reaches.dataset)
ridmap = rmap.get_centerlines_map()
ridmap

## Map by algorithm

In [ ]:
#hidvi
Qbar_hi=dict()
for reachid in reachids_oh:    
    idx = np.where(results['reaches']['reach_id'][:] == np.int64(reachid) )
    data = np.ma.getdata(results["hivdi"]["Q"][idx])[0]
    if data.max() > 0:
        Qbar_hi[reachid] = 1
    else:
        Qbar_hi[reachid] = 0
        
#momma
Qbar_momma=dict()
for reachid in reachids_oh:    
    idx = np.where(results['reaches']['reach_id'][:] == np.int64(reachid) )
    data = np.ma.getdata(results["momma"]["Q"][idx])[0]
    if data.max() > 0:
        Qbar_momma[reachid] = 1
    else:
        Qbar_momma[reachid] = 0
        
#sad
Qbar_sad=dict()
for reachid in reachids_oh:    
    idx = np.where(results['reaches']['reach_id'][:] == np.int64(reachid) )
    data = np.ma.getdata(results["sad"]["Qa"][idx])[0]
    if data.max() > 0:
        Qbar_sad[reachid] = 1
    else:
        Qbar_sad[reachid] = 0
        
        
#sic4dvar
Qbar_sic=dict()
for reachid in reachids_oh:    
    idx = np.where(results['reaches']['reach_id'][:] == np.int64(reachid) )
    data = np.ma.getdata(results["sic4dvar"]["Q_da"][idx])[0]
    #print(data)
    if data.max() > 0:
        Qbar_sic[reachid] = 1
    else:
        Qbar_sic[reachid] = 0

In [ ]:
rmap._dataset['HiVDI']=-1.

for reachid in reachids_oh:
    if not np.isnan(Qbar_hi[str(reachid)]):
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['HiVDI']]=Qbar_hi[str(reachid)]
    

rmap._json_dataset = rmap._dataset.to_json()    
hi_map = rmap.get_centerlines_map(varname="HiVDI",varlimits=[0,1],cmap=['r','b'])
hi_df = pd.Series(Qbar_hi)
print(len(hi_df[hi_df>0]))
hi_map

In [ ]:
# add integrated MetroMan discharge to the rmap object
rmap._dataset['Momma']=-1.

for reachid in reachids_oh:
    if not np.isnan(Qbar_momma[str(reachid)]):
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['Momma']]=Qbar_momma[str(reachid)]
rmap._json_dataset = rmap._dataset.to_json()
momma_map = rmap.get_centerlines_map(varname="Momma",varlimits=[0,1],cmap=['r','b'])
momma_df = pd.Series(Qbar_momma)
print(len(momma_df[momma_df>0]))
momma_map

In [ ]:
rmap._dataset['Sad']=-1.

for reachid in reachids_oh:
    if not np.isnan(Qbar_sad[str(reachid)]):
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['Sad']]=Qbar_sad[str(reachid)]
rmap._json_dataset = rmap._dataset.to_json()
sad_map = rmap.get_centerlines_map(varname="Sad",varlimits=[0,1],cmap=["r", "b"])
sad_df = pd.Series(Qbar_sad)
print(len(sad_df[sad_df>0]))
sad_map

In [ ]:
rmap._dataset['Sic']=-1.

for reachid in reachids_oh:
    if not np.isnan(Qbar_sic[str(reachid)]):
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['Sic']]=Qbar_sic[str(reachid)]
rmap._json_dataset = rmap._dataset.to_json()
sic_map = rmap.get_centerlines_map(varname="Sic",varlimits=[0,1],cmap=["r", "b"])
sic_df = pd.Series(Qbar_sic)
print(len(sic_df[sic_df>0]))
sic_map

In [ ]:
#Map all algorithms

Qbars,algo = {},{}
for reachid in reachids_oh:    
    idx = np.where(results['reaches']['reach_id'][:] == np.int64(reachid) )
    counter = []
    temp_algo = ""
    if np.ma.getdata(results["hivdi"]["Q"][idx])[0].max() > 0:
        counter.append(1)
        temp_algo = f"hivdi"
    if np.ma.getdata(results["momma"]["Q"][idx])[0].max() > 0:
        counter.append(1)
        temp_algo = f"{temp_algo}, momma"
    if np.ma.getdata(results["sad"]["Qa"][idx])[0].max() > 0:
        counter.append(1)
        temp_algo = f"{temp_algo}, sad"
    if np.ma.getdata(results["sic4dvar"]["Q_da"][idx])[0].max() > 0:
        counter.append(1)
        temp_algo = f"{temp_algo}, sic4dvar"
        
    Qbars[reachid] = sum(counter)
    algo[reachid] = temp_algo


In [ ]:
rmap._dataset['all']=-1.

for reachid in reachids_oh:
    if not np.isnan(Qbars[str(reachid)]):
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['all']]=Qbars[str(reachid)]
    #if not algo[str(reachid)]:
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['algo']]=algo[str(reachid)]
rmap._dataset['wse'] = np.round(rmap._dataset['wse'],2)
rmap._json_dataset = rmap._dataset.to_json()
all_map = rmap.get_centerlines_map(varname="all",varlimits=[0,4],cmap=["red","orange", "yellow", "green", "blue"]
                                   ,tooltip_attributes=['reach_id','all','algo','wse','width','river_name']) 
all_map

## Close output file

In [ ]:
results.close()